In [ ]:
import pandas as pd
from collections import defaultdict
import ast
from copy import deepcopy
import json

In [ ]:
# Read the datasets

amazon_books = pd.read_csv('amazon_books_data.csv')
amazon_reviews = pd.read_csv('amazon_books_rating.csv')
goodreads_books = pd.read_csv('goodreads_books.csv')

In [ ]:
amazon_books.columns

In [ ]:
amazon_books = amazon_books.drop(columns=["ratingsCount", "infoLink"])
amazon_books.columns

In [ ]:
amazon_reviews.columns

In [ ]:
amazon_reviews = amazon_reviews.drop(columns=["Id", "User_id", "profileName", "review/helpfulness","review/time","review/summary"])
amazon_reviews.columns

In [ ]:
goodreads_books.columns

In [ ]:
goodreads_books = goodreads_books.drop(columns=["bookformat", "isbn","isbn13", "reviews","totalratings"])
goodreads_books.columns

In [ ]:
amazon_books_dict = amazon_books.groupby('Title').apply(lambda x: x.to_dict(orient='records')).to_dict()

In [ ]:
amazon_reviews_dict = amazon_reviews.groupby('Title').apply(lambda x: x.to_dict(orient='records')).to_dict()

In [ ]:
goodreads_books_dict = goodreads_books.groupby('title').apply(lambda x: x.to_dict(orient='records')).to_dict()

In [ ]:
count = 5
print("\n")
print("Sample Amazon Books:")
for title, entries in list(amazon_books_dict.items())[:count]:
    print(f"Title: {title}")
    print(entries)
    print("\n")

print("\n")
print("Sample Amazon Reviews:")
for title, entries in list(amazon_reviews_dict.items())[:count]:
    print(f"Title: {title}")
    print(entries)
    print("\n")

print("\n")
print("Sample Goodreads Books:")
for title, entries in list(goodreads_books_dict.items())[:count]:
    print(f"Title: {title}")
    print(entries)
    print("\n")

In [ ]:
amazon_reviews_dict["Dr. Seuss: American Icon"]

In [ ]:

nan = float('nan')
def clean_up_amazon_reviews_dict(reviews):
    new_amazon_reviews_dict = {}
    for book_title, entries in reviews.items():
        title = book_title
        price = 0
        avg_rating = 0
        reviews = []
        for entry in entries:
            p = entry["Price"]
            rat = entry["review/score"]
            rev = entry["review/text"]

            rat, rev
            if p is not nan:
                price = p
            if rat is not nan:
                avg_rating += rat 
            reviews.append(rev)

        avg_rating = round(avg_rating / len(entries), 1)
        if price == 0:
            price = ""

        if avg_rating == 0:
            avg_rating = ""
        
        new_amazon_reviews_dict[title] = {
            "title": title,
            "price": price,
            "avg_rating": avg_rating,
            "reviews": reviews
        }
    return new_amazon_reviews_dict
new_amazon_reviews_dict = clean_up_amazon_reviews_dict(deepcopy(amazon_reviews_dict))
new_amazon_reviews_dict



In [ ]:
new_amazon_reviews_dict["Dr. Seuss: American Icon"]

In [ ]:
print(amazon_books_dict["Death Dream"][0]["categories"])
print(amazon_books_dict["Dr. Seuss: American Icon"][0]["categories"])

In [ ]:
# Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
#        'publishedDate', 'categories'],
#       dtype='object')
def clean_up_categories(categories_str):
    if not categories_str:
        return []
    
    open_bracket = categories_str.index("[")
    close_bracket = categories_str.index("]")
    categories_str = categories_str[open_bracket + 1: close_bracket]

    ampersand = categories_str.index()


def clean_up_amazon_books_dict(books):
    new_amazon_books_dict = {}
    for title, entry in books.items():
        entry = entry[0]

        entry["title"] = title
        del entry["Title"]

        # categs_list = ast.literal_eval(entry["categories"])
        # entry["categories"] = categs_list
        
        new_amazon_books_dict[title] = entry
    return new_amazon_books_dict
new_amazon_books_dict = clean_up_amazon_books_dict(deepcopy(amazon_books_dict))

In [ ]:
new_amazon_books_dict["Dr. Seuss: American Icon"]

In [ ]:
amazon_books_titles = set(amazon_books_dict.keys())
goodreads_books_titles = set(goodreads_books_dict.keys())
similar = amazon_books_titles.intersection(goodreads_books_titles)
print(f"Number of amazon books: {len(amazon_books_titles)}\n Number of goodreads books: {len(goodreads_books_titles)} \n Number of books in both: {len(similar)}")

In [ ]:
titles_from_books = set(new_amazon_books_dict.keys())
titles_from_reviews = set(new_amazon_reviews_dict.keys())
similar = titles_from_reviews.intersection(titles_from_books)
print(f"Number of amazon books titles: {len(titles_from_books)}\n Number of amazon reviews titles : {len(titles_from_reviews)} \n Number of books in both: {len(similar)}")

In [ ]:
complete_dataset = {}
for title in new_amazon_books_dict.keys():
    book_data = new_amazon_books_dict[title]
    review_data = new_amazon_reviews_dict[title]
    complete_dataset[title] = (book_data | review_data)

In [ ]:
complete_dataset["Dr. Seuss: American Icon"]

In [ ]:
def remove_square_brackets(str):
    open_bracs = str.find("[")
    close_bracs = str.find("]")
    return str[open_bracs + 2: close_bracs - 1]

def convert_string_to_list(str_list):
    if not isinstance(str_list, str) or str_list == "" :
        return []
    str_list = remove_square_brackets(str_list)
    return str_list.split(",")

def convert_authors_and_categs_to_list(dataset):
    for _, record in dataset.items():
        categs = convert_string_to_list(record["categories"])
        authors = convert_string_to_list(record["authors"])
        record["categories"] = categs
        record["authors"] = authors
    return dataset

complete_dataset = convert_authors_and_categs_to_list(deepcopy(complete_dataset))

In [ ]:
(complete_dataset["Eyewitness Travel Guide to Europe"])


In [ ]:
dataset_json = json.dumps(complete_dataset, indent=4)

In [ ]:
#print(dataset_json)

In [ ]:
# with open("dataset.json", "w", encoding="utf-8") as file:
#     file.write(dataset_json)

In [ ]:
list(complete_dataset.values())[0]

In [ ]:
complete_books_dataset = {}
complete_reviews_dataset = {}

for title, record in complete_dataset.items():
    complete_reviews_dataset[title] = record["reviews"]

    new_record = deepcopy(record)
    del new_record["reviews"]
    complete_books_dataset[title] = new_record


In [ ]:
books_json = json.dumps(complete_books_dataset, indent=4)
with open("books.json", "w", encoding="utf-8") as file:
    file.write(books_json)

# reviews_json = json.dumps(complete_reviews_dataset, indent=4)
# with open("reviews.json", "w", encoding="utf-8") as file:
#     file.write(reviews_json)

In [ ]:
[       "description", "authors", 
        "image",
        "previewLink",
        "publisher",
        "publishedDate",
        "categories" ,
        "title",
        "price",
        "avg_rating"]